In [ ]:
'''
Clayton Cohn
20200217
CSC440
Assignment 4
'''

'''
1. (10 points) Exercise 4, page 147

    For a string of bits S, let Sbar denote the complementary string obtained by changing all the 1s to 0s
    and all the 0s to 1s (equivalently, Sbar = S ^ 11111....). Show that if DES key K encrypts P to C, then
    Kbar encrypts Pbar to Cbar. (Hint: This has nothing to do with the structure of the S-boxes. To do the
    problem, just work through the encryption algorithm). 
'''

In [ ]:
'''
2. (10 points) Exercise 6 on page 147

    Suppose Triple DES is performed by choosing two keys K1, K2 and computing Ek1(Ek2(Ek2(m))) (note that the order
    of the keys has been modified from the usual two-key version of Triple DES). Show how to attack this modified 
    version with a meet-in-the-middle attack.
'''

In [181]:
'''
3. (50 points) Implement encryption in the textbook's simplified DES scheme. 

    Write the program so that it contains a method called encrypt(plaintext, key) 
    where the plaintext and the key are both integers. The first must be in the range 0 to 4095 
    and the second in the range 0 to 511. 
    
    My suggestion is to write this method with the first part 
    generating a list or array of the four sub-keys and the second part a loop that iterates four times, 
    once for each round. In that loop, call methods for carrying out an exclusive-or, for expanding, 
    and for applying each of the two S-boxes. I am leaving a number of details unspecified but those will 
    be filled in as you begin to work on this and ask me questions. I will provide test data in the form 
    of plaintexts, keys, and their resulting ciphertexts.

    Please also add to the program a driver method for testing your encryption. This method should prompt 
    for the plaintext and the key as integers, call the encryption method, and then output the ciphertext.
'''

import binascii
from textwrap import wrap

def encrypt(plaintext, key):
    
    plaintext = bin(plaintext)[2:]
    key = bin(key)[2:]
    
    print("\nPlaintext: {0}".format(plaintext))
    print("\nKey: {0}".format(key))
    
    # First make sure arguments are without bounds
    if len(plaintext) not in range(0, 4096) or len(key) not in range(0, 512):
        print("Invalid lengths.")
        return
    
    # Identify the s-boxes
    s1 =    [
                ["101", "010", "001", "110", "011", "100", "111", "000"],
                ["001", "100", "110", "010", "000", "111", "101", "011"] 
            ]
    s2 =    [
                ["100", "000", "110", "101", "111", "001", "011", "010"],
                ["101", "011", "000", "111", "110", "010", "001", "100"]
            ]
    
    # Separate plaintext into 12-bit blocks
    blocks = wrap(plaintext,12)
    
    # Pad last block with 0s
    blocks[-1] = blocks[-1].ljust(12,"0")[:12]
    
    # Generate subkeys from key
    if len(key) == 0: key = "0"
    long_key = key * 11
    subkeys = [long_key[0:8], long_key[1:9], long_key[2:10], long_key[3:11]]
    print("\nSubkeys: {}".format(subkeys))
    
    print("\nBlocks: {0}".format(blocks))
    
    # Create ciphertext
    ciphertext = ""
    
    # For each block...
    for i in range(0,len(blocks)):
        block = blocks[i]
        
        # Split block into left and right
        l, r = block[:len(block)//2], block[len(block)//2:]
        print("\nBlock {0}\nLeft: {1}\nRight: {2}".format(i,l,r))
        
        # 4 rounds for simplified version
        for j in range (0,4):
            
            # Copy previous left and right
            l_old = l
            
            # Get appropriate key
            k = subkeys[j]
            
            # Assign next l to current r
            l = r
            
            # Perform f on r
        
            # First expand the r
            r = expand(r)
            
            # XOR k (keys[i]) with r
            r = xor(r,k,8)
            
            # Split bits and prepare for s-boxes
            r1, r2 = r[:len(r)//2], r[len(r)//2:] 
            print("r1: " + r1)
            print("r2: " + r2)
            
            s1row = int(r1[0])
            s1col = int(r1[1:],2)
            print("S1 row: {0}, S1 col: {1}".format(s1row,s1col))
    
            s2row = int(r2[0])
            s2col = int(r2[1:],2)
            print("S2 row: {0}, S2 col: {1}".format(s2row,s2col))
            
            sbox1_choice = s1[s1row][s1col]
            sbox2_choice = s2[s2row][s2col]
            print("S1 choice: {0}, S2 choice: {1}".format(sbox1_choice,sbox2_choice))
            
            #Output of function f is those 6 bits
            f = sbox1_choice + sbox2_choice
            
            # To get the new r, we need to xor f with the previous l
            r = xor(l_old,f,6)
            print("END Block {0} - Round {1}: L = {2}, R = {3}".format(i,j,l,r))
            
            print("\n")
        # Add blocks to ciphertext
        ciphertext += l + r
    
    print("Final ciphertext: " + ciphertext)
    if len(ciphertext) == 0: return 0
    else: ciphertext = int(ciphertext, 2)
        
    # Return int ciphertext
    # print("\nCiphertext: {0}".format(ciphertext))
    return ciphertext
        
# Helper function to use as E for expansion from 6 bits to 8
def expand(s):
    result = s[0] + s[1] + s[3] + s[2] + s[3] + s[2] + s[4] + s[5]
    print(s + " expanded to " + result)
    return result

# Helper function to xor two bit strings
def xor(s1, s2, n):
    n1 = int(s1,2)
    n2 = int(s2,2)
    result = bin(n1^n2)[2:]
    
    # If result is not 8 bits, we must 0 append to beginning of result
    nZeros = n-len(result)
    zs = "0" * nZeros
    result = zs + result
    
    print(s1 + " XOR " +  s2 + " = " + result)
    return result
    

In [183]:
encrypt(0, 368)


Plaintext: 100010110101

Key: 111000111

Subkeys: ['11100011', '11000111', '10001111', '00011111']

Blocks: ['100010110101']

Block 0
Left: 100010
Right: 110101
110101 expanded to 11101001
11101001 XOR 11100011 = 00001010
r1: 0000
r2: 1010
S1 row: 0, S1 col: 0
S2 row: 1, S2 col: 2
S1 choice: 101, S2 choice: 000
100010 XOR 101000 = 001010
END Block 0 - Round 0: L = 110101, R = 001010


001010 expanded to 00010110
00010110 XOR 11000111 = 11010001
r1: 1101
r2: 0001
S1 row: 1, S1 col: 5
S2 row: 0, S2 col: 1
S1 choice: 111, S2 choice: 000
110101 XOR 111000 = 001101
END Block 0 - Round 1: L = 001010, R = 001101


001101 expanded to 00111101
00111101 XOR 10001111 = 10110010
r1: 1011
r2: 0010
S1 row: 1, S1 col: 3
S2 row: 0, S2 col: 2
S1 choice: 010, S2 choice: 110
001010 XOR 010110 = 011100
END Block 0 - Round 2: L = 001101, R = 011100


011100 expanded to 01111100
01111100 XOR 00011111 = 01100011
r1: 0110
r2: 0011
S1 row: 0, S1 col: 6
S2 row: 0, S2 col: 3
S1 choice: 111, S2 choice: 101
00110

1840